TRABAJO FINAL

In [8]:
import networkx as nx
import pandas as pd
import folium
from IPython.display import display
import tkinter as tk
from tkinter import messagebox


def encontrar_ruta_mas_corta(ciudad, inicio, fin):
    ruta_mas_corta = nx.shortest_path(ciudad, inicio, fin, weight='distancia')
    return ruta_mas_corta


# Leer el archivo nodos.csv
nodos_df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vQVvI8AJiIcGjl9TT7HLlvk0zuhLyGcQN-uh0EZd3X388zbO2L4G3h_GdAIzgG7ba0yy72kacRgvJw6/pub?output=csv')

# Crear el grafo
G = nx.Graph()

# Agregar los nodos al grafo con sus coordenadas
for i, nodo in nodos_df.iterrows():
    G.add_node(nodo['Nodo'], pos=(nodo['Latitud'], nodo['Longitud']))

# Leer el archivo aristas.csv
aristas_df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vQAETHQL3siWVtacszSu2oU-fFIiDmssF5O7PkokJT756442HA4iiGa-7c3gneUD102UyOE75BPfCV9/pub?output=csv')

# Agregar las aristas al grafo con sus distancias
for i, arista in aristas_df.iterrows():
    G.add_edge(arista['Nodo_A'], arista['Nodo_B'], weight=arista['Distancia'])


def buscar_ruta():
    try:
        start_node = int(start_entry.get())
        end_node = int(end_entry.get())

        # Lista para almacenar las rutas encontradas
        all_paths = []

        # Encontrar las rutas para el par de nodos de inicio y fin ingresados
        paths = list(nx.all_simple_paths(G, start_node, end_node, cutoff=20))  # Establecer el límite en 10
        if len(paths) > 5:
            paths = paths[:5]  # Limitar el número de rutas a 5
        all_paths.extend(paths)

        # Crear un mapa con folium
        mapa = folium.Map(location=[nodos_df['Latitud'].mean(), nodos_df['Longitud'].mean()], zoom_start=15)

        # Agregar los nodos de interés al mapa
        for i, nodo in nodos_df.iterrows():
            if nodo['Nodo'] == start_node or nodo['Nodo'] == end_node:
                color = 'green' if nodo['Nodo'] == start_node else 'red'
                label = 'Inicio' if nodo['Nodo'] == start_node else 'Fin'
                folium.CircleMarker(
                    location=[nodo['Latitud'], nodo['Longitud']],
                    radius=4,
                    color='black',
                    fill=True,
                    fill_color=color
                ).add_to(mapa)
                folium.Marker(
                    location=[nodo['Latitud'], nodo['Longitud']],
                    tooltip=f"{label}: Nodo {nodo['Nodo']}",  # Agrega el texto "Inicio" o "Fin" como tooltip
                    icon=folium.Icon(icon='info-sign', prefix='fa', color='blue')
                ).add_to(mapa)

        # Agregar las aristas al mapa
       # for edge in G.edges():
        #    nodo_a = nodos_df.loc[nodos_df['Nodo'] == edge[0]]
         #   nodo_b = nodos_df.loc[nodos_df['Nodo'] == edge[1]]
          #  folium.PolyLine(
           #     locations=[(nodo_a['Latitud'].values[0], nodo_a['Longitud'].values[0]),
            #               (nodo_b['Latitud'].values[0], nodo_b['Longitud'].values[0])],
             #   color='gray',
              #  weight=2,
               # opacity=0.7
            #).add_to(mapa)

        # Dibujar las rutas con colores y grosores diferentes
        colors = ['black', 'green', 'orange', 'purple', 'blue']  # Colores para las rutas
        thickness_values = [10, 8, 6, 4, 2]  # Valores de grosor

        for i, path in enumerate(all_paths):
            if i >= len(thickness_values):
                thickness = thickness_values[-1]  # Si se excede la cantidad de valores de grosor, se usa el último valor
            else:
                thickness = thickness_values[i]

            edges = list(zip(path[:-1], path[1:]))
            for edge in edges:
                nodo_a = nodos_df.loc[nodos_df['Nodo'] == edge[0]]
                nodo_b = nodos_df.loc[nodos_df['Nodo'] == edge[1]]
                folium.PolyLine(
                    locations=[(nodo_a['Latitud'].values[0], nodo_a['Longitud'].values[0]),
                               (nodo_b['Latitud'].values[0], nodo_b['Longitud'].values[0])],
                    color=colors[i % len(colors)],
                    weight=thickness,
                    opacity=1.0
                ).add_to(mapa)
                
        
        # Mostrar los detalles de las rutas
        message = ""
        for i, path in enumerate(all_paths):
            message += f"Ruta {i+1}:\n"
            total_distance = 0
            route = ""
            for j in range(len(path) - 1):
                node_a = path[j]
                node_b = path[j + 1]
                distance = G.get_edge_data(node_a, node_b)['weight']
                route += f"{node_a} --({distance})--> "
                total_distance += distance
            route += f"{path[-1]}\n"  # Agregar el último nodo al recorrido
            message += f"Recorrido de la ruta {i+1}: {route}"
            message += f"Distancia total de la ruta {i+1}: {total_distance}\n\n"

        # Encontrar la ruta más corta
        ruta_mas_corta = encontrar_ruta_mas_corta(G, start_node, end_node)
        # Resaltar la ruta más corta en el mapa
        for i in range(len(ruta_mas_corta)-1):
            nodo_actual = ruta_mas_corta[i]
            nodo_siguiente = ruta_mas_corta[i+1]
            folium.PolyLine([(nodos_df.loc[nodos_df['Nodo'] == nodo_actual]['Latitud'].values[0], nodos_df.loc[nodos_df['Nodo'] == nodo_actual]['Longitud'].values[0]),
                            (nodos_df.loc[nodos_df['Nodo'] == nodo_siguiente]['Latitud'].values[0], nodos_df.loc[nodos_df['Nodo'] == nodo_siguiente]['Longitud'].values[0])], color='red').add_to(mapa)

        distancia_total = 0
        nodos_recorridos = []
        ruta_str = ""
        for i in range(len(ruta_mas_corta)-1):
            nodo_actual = ruta_mas_corta[i]
            nodo_siguiente = ruta_mas_corta[i+1]
            distancia_arista = G.get_edge_data(nodo_actual, nodo_siguiente)['weight']
            distancia_total += distancia_arista
            nodos_recorridos.append(nodo_actual)
            ruta_str += f"{nodo_actual} --({distancia_arista})--> "
        nodos_recorridos.append(ruta_mas_corta[-1])
        ruta_str += f"{ruta_mas_corta[-1]}"
        message += f"Ruta más corta:\n"
        message += f"Recorrido de la ruta más corta: {ruta_str}\n"
        message += f"Distancia total de la ruta más corta: {distancia_total}\n"

        # Mostrar el mapa
        display(mapa)

        # Mostrar los detalles de las rutas en una ventana emergente
        messagebox.showinfo("Detalles de las rutas", message)

        # Guardar el mapa como archivo HTML
        mapa.save('ruta.html')
        messagebox.showinfo("Archivo HTML generado", "El archivo 'ruta.html' se ha generado exitosamente.")
    except Exception as e:
        messagebox.showerror("Error", "Ocurrió un error al buscar la ruta. Por favor, verifica los nodos de inicio y fin ingresados.")

# Crear la ventana principal
window = tk.Tk()
window.title("Búsqueda de rutas")

# Crear los elementos de la interfaz gráfica
start_label = tk.Label(window, text="Nodo de inicio:")
start_label.pack()
start_entry = tk.Entry(window)
start_entry.pack()

end_label = tk.Label(window, text="Nodo de fin:")
end_label.pack()
end_entry = tk.Entry(window)
end_entry.pack()

search_button = tk.Button(window, text="Buscar ruta", command=buscar_ruta)
search_button.pack()

# Ejecutar la ventana principal
window.mainloop()
